In [1]:
import pandas as pd
import ast
from IPython.display import display
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm

In [2]:
class CFG:
    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

In [3]:
annotations=pd.read_csv(f"{CFG.dataset_dir}train_annotations.csv")
annotatin_cvc=annotations[annotations["label"].str.contains("CVC")]
print(len(annotatin_cvc))

11629


In [4]:
annotatin_cvc

,StudyInstanceUID,label,data
0,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1487, 1279], [1477, 1168], [1472, 1052], [14..."
1,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1328, 7], [1347, 101], [1383, 193], [1400, 2..."
2,1.2.826.0.1.3680043.8.498.72921907356394389969...,CVC - Borderline,"[[801, 1207], [812, 1112], [823, 1023], [842, ..."
3,1.2.826.0.1.3680043.8.498.11697104485452001927...,CVC - Normal,"[[1366, 961], [1411, 861], [1453, 751], [1508,..."
5,1.2.826.0.1.3680043.8.498.87704688663091069148...,CVC - Normal,"[[906, 604], [1103, 578], [1242, 607], [1459, ..."
...,...,...,...
17993,1.2.826.0.1.3680043.8.498.88885675487819792225...,CVC - Normal,"[[1384, 1098], [1480, 899], [1554, 774], [1630..."
17995,1.2.826.0.1.3680043.8.498.18590666552942008211...,CVC - Normal,"[[1250, 1105], [1247, 1031], [1248, 970], [126..."
17996,1.2.826.0.1.3680043.8.498.66774579023475418162...,CVC - Normal,"[[1379, 1402], [1389, 1233], [1386, 1137], [13..."
17997,1.2.826.0.1.3680043.8.498.18560506324214035535...,CVC - Normal,"[[1302, 1543], [1308, 1488], [1317, 1355], [13..."


In [5]:
def count_cvc_per_uid():
    uid_count_list=[]
    for uid in tqdm(annotatin_cvc["StudyInstanceUID"].unique()):
        uid_count_list.append((uid,len(annotatin_cvc[annotatin_cvc["StudyInstanceUID"]==uid])))

    uid_count=pd.DataFrame(uid_count_list,columns=["StudyInstanceUID","count"])
    return uid_count

In [20]:
train=pd.read_csv(f"{CFG.dataset_dir}train.csv")
end_points=pd.read_csv("cvc_end_points.csv")

,StudyInstanceUID,x,y
0,1.2.826.0.1.3680043.8.498.12616281126973421762...,1487,1279
1,1.2.826.0.1.3680043.8.498.12616281126973421762...,1457,1006
2,1.2.826.0.1.3680043.8.498.72921907356394389969...,801,1207
3,1.2.826.0.1.3680043.8.498.11697104485452001927...,1366,961
4,1.2.826.0.1.3680043.8.498.87704688663091069148...,1482,1225
...,...,...,...
11624,1.2.826.0.1.3680043.8.498.88885675487819792225...,1384,1098
11625,1.2.826.0.1.3680043.8.498.18590666552942008211...,1250,1105
11626,1.2.826.0.1.3680043.8.498.66774579023475418162...,1379,1402
11627,1.2.826.0.1.3680043.8.498.18560506324214035535...,1302,1543


In [29]:
def visualize_annotations(uid):
    rows=end_points[end_points["StudyInstanceUID"]==uid]
    annot_data_list=[]
    for idx,row in rows.iterrows():
        # annot_data=ast.literal_eval(row["data"])
        # annot_data=np.array(annot_data)
        annot_data=np.array((row["x"],row["y"]))
        annot_data_list.append(annot_data)

    image = cv2.imread(f"{CFG.dataset_dir}train/{uid}.jpg")
    image=np.zeros(image.shape,dtype=np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    color_list=["b","g","r","c","m","y","k","w"]
    for i,annot_data in enumerate(annot_data_list):
        image=cv2.circle(image,tuple(annot_data),radius=30,color=(255,255,255),thickness=-1)

    dir_path="../input/cvc_annotated_cv2"
    os.makedirs(dir_path,exist_ok=True)
    cv2.imwrite(f"{dir_path}/{uid}.jpg",image)


def get_end_point(uid,annot_data):
    image = cv2.imread(f"{CFG.dataset_dir}train/{uid}.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    center=np.array([image.shape[0]/2,image.shape[1]/2])
    first_point=annot_data[0]
    last_point=annot_data[-1]
    if np.linalg.norm(first_point-center)>np.linalg.norm(last_point-center):
        end_point=last_point
    else:
        end_point=first_point
    return end_point


def visualize_annotations_line(uid):
    rows=annotatin_cvc[annotatin_cvc["StudyInstanceUID"]==uid]
    annot_data_list=[]
    for idx,row in rows.iterrows():
        annot_data=ast.literal_eval(row["data"])
        annot_data=np.array(annot_data)
        # annot_data=np.array((row["x"],row["y"]))
        annot_data_list.append(annot_data)

    image = cv2.imread(f"{CFG.dataset_dir}train/{uid}.jpg")
    image=np.zeros(image.shape,dtype=np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.polylines(image,annot_data_list,color=(255,255,255),isClosed=False,thickness=15)

    dir_path="../input/cvc_annotated_line_mask"
    os.makedirs(dir_path,exist_ok=True)
    cv2.imwrite(f"{dir_path}/{uid}.jpg",image)

In [30]:
for uid in tqdm(annotatin_cvc["StudyInstanceUID"].unique()):
    visualize_annotations_line(uid)

100%|██████████| 8853/8853 [20:51<00:00,  7.07it/s]
